In [1]:
# Standard library imports
import os
import requests

# Third party imports
import ee
import ee.mapclient
import folium
import geopandas as gpd
from google.oauth2 import service_account
from osgeo import gdal
from pyproj import Transformer

from dotenv import load_dotenv

#Specify the path to the service account key file (found in .env)
load_dotenv()
key_path = os.getenv('GEE_AUTH_KEY_PATH')

# Define the correct scope for Earth Engine
SCOPES = ['https://www.googleapis.com/auth/earthengine']

# Load the service account credentials
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=SCOPES
)

# Initialize Earth Engine
ee.Initialize(credentials=credentials)
print("Earth Engine initialized successfully!")

Earth Engine initialized successfully!


In [2]:
roi = 'data/vector/all_areas.geojson'
buffer_amt = 2500

# Read the ROI file
gdf = gpd.read_file(roi)

# Create a buffered envelope (bounding box)
buffered_bbox = gdf.envelope.buffer(buffer_amt, cap_style=3)  # cap_style=3 creates square buffers

# Transform to EPSG:4326 for Earth Engine
buffered_bbox_4326 = buffered_bbox.to_crs("EPSG:4326")

# Convert the GeoSeries to a GeoDataFrame
buffered_bbox_gdf = gpd.GeoDataFrame(geometry=buffered_bbox_4326)

# Convert all geometries to Earth Engine geometries
ee_geometries = []
for _, row in buffered_bbox_gdf.iterrows():
    coords = row.geometry.exterior.coords.xy
    ee_coords = [[x, y] for x, y in zip(coords[0], coords[1])]
    ee_geometries.append(ee.Geometry.Polygon(ee_coords, proj='EPSG:4326'))

# Create a GeometryCollection from the list of geometries
geometry_collection = ee.Geometry.MultiPolygon(ee_geometries)

# Get the bounding box of the GeometryCollection
bounding_box = geometry_collection.bounds()

# Load the National Elevation Dataset (NED)
ned = ee.Image("USGS/3DEP/10m")

# Clip the NED to the bounding box
ned_clipped = ned.clip(bounding_box)

In [3]:

# Define visualization parameters
vis_params = {'min': 1000, 'max': 1500, 'palette': ['440154', '482878', '3E4A89', '31688E', '26828E', '1F9E89', '35B779', '6DCD59', 'B4DE2C', 'FDE725']}

# Create a folium map object.
# Instead of using centroid, use the bounding box coordinates directly
bounding_box_coords = bounding_box.coordinates().getInfo()[0]
# Calculate the center of the bounding box
lon_coords = [coord[0] for coord in bounding_box_coords]
lat_coords = [coord[1] for coord in bounding_box_coords]
centroid_lon = (min(lon_coords) + max(lon_coords)) / 2
centroid_lat = (min(lat_coords) + max(lat_coords)) / 2


m = folium.Map(location=[centroid_lat, centroid_lon], zoom_start=12)

# Add the NED layer to the map
map_id_dict = ned_clipped.getMapId(vis_params)
folium.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NED',
).add_to(m)


# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map
display(m)

In [4]:
output_dir = 'data/raster/ned'
os.makedirs(output_dir, exist_ok=True)

# Set the GTIFF_SRS_SOURCE configuration option to EPSG
gdal.SetConfigOption('GTIFF_SRS_SOURCE', 'EPSG')

# Choose a value that doesn't exist in your data and doesn't have meaning in the context of your data.
# Convert to int16 and set unmask value
remapped_image = ned_clipped.unmask(0.0)

# Set an explicit "no data" value to null in the download URL parameters
url = remapped_image.getDownloadURL({
    'scale': 30,
    'crs': 'EPSG:6514',
    'region': bounding_box,
    'format': 'GeoTIFF',
    'formatOptions': {
        'cloudOptimized': True,
        'noData': 0.0
    }
})

response = requests.get(url)

# Create output file path
output_file = os.path.join(output_dir, f"ned.tif")

# Write response content directly to final file
with open(output_file, 'wb') as f:
    f.write(response.content)

print(f"Downloaded file {output_file}")

# Set nodata value to 9999
dataset = gdal.Open(output_file, gdal.GA_Update)
band = dataset.GetRasterBand(1)
band.SetNoDataValue(0.0)
dataset = None

Downloaded file data/raster/ned/ned.tif


/opt/miniconda3/envs/calibrate-rap/lib/python3.10/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported
